# Install

In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets
!pip install -q -U rouge

# Import

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

# Load Quantized Model

In [ ]:
# model_id = "tiiuae/falcon-7b-instruct" # Session crashed after using all available RAM.
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Train Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import pandas as pd
# import yaml

# def load_yaml_documents(file_path):
#     with open(file_path, 'r') as file:
#         documents = file.read().replace('!!python/object:gqa.types.DocumentSpec', '').replace('!!python/object:gqa.types.GraphSpec', '').replace('!!python/object:gqa.types.QuestionSpec', '').replace('!!python/object:gqa.types.NodeSpec', '').replace('!!python/object:gqa.types.EdgeSpec', '').replace('!!python/object:gqa.types.LineSpec', '').split('---')
#         data = [yaml.load(doc, Loader=yaml.FullLoader) for doc in documents if doc.strip()]
#     return data

# file_path = '/content/drive/MyDrive/Graph q&a data/gqa-r0.yaml'
# data = load_yaml_documents(file_path)

# # Assuming that each document in the YAML file has the structure: {'answer': ..., 'graph': {...}, 'question': {...}}
# # You can extract the 'answer', 'graph', and 'question' fields and create a DataFrame.

# df = pd.DataFrame(data, columns=['answer', 'graph', 'question'])

# # Now df is a pandas DataFrame with three columns: 'answer', 'graph', and 'question'.


In [ ]:
# df.to_csv('/content/drive/MyDrive/Graph q&a data/cypher_Q&A.csv')

In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/Graph q&a data/cypher_Q&A.csv', index_col=0)

In [ ]:
# df.shape

In [ ]:
# df.head(1)

In [ ]:
# eval(df.question[0]).keys()

In [ ]:
# eval(df.graph[0]).keys()

In [ ]:
# eval(df.question[0])['english']

In [ ]:
# eval(df.question[0])['cypher']

In [ ]:
# eval(df.graph[0])['edges'][0]

In [ ]:
# eval(df.graph[0])['nodes'][0]

In [ ]:
# df.shape[0]

In [ ]:
# nodes = []
# edges = []
# english = []
# cypher = []

# for i in range(0,df.shape[0]):
#     nodes.append(eval(df.graph[i])['nodes'])
#     edges.append(eval(df.graph[i])['edges'])
#     english.append(eval(df.question[i])['english'])
#     cypher.append(eval(df.question[i])['cypher'])

In [ ]:
# nodes

In [ ]:
# english

In [ ]:
# lol_df = pd.DataFrame()
# lol_df['nodes'] = nodes
# lol_df['edges'] = edges
# lol_df['english'] = english
# lol_df['cypher'] = cypher

In [ ]:
# lol_df.head()

In [ ]:
# lol_df.to_csv('/content/drive/MyDrive/Graph q&a data/final_cypher_Q&A.csv')

In [ ]:
import pandas as pd
lol_df = pd.read_csv('/content/drive/MyDrive/cypher_Q&A/final_cypher_Q&A.csv', index_col=0)

In [ ]:
lol_df = lol_df.sample(frac=1).reset_index(drop=True)
lol_df.head()

,nodes,edges,english,cypher
0,"[{'architecture': 'art-deco', 'cleanliness': '...","[{'line_color': 'gray', 'line_id': '8d8e8f81-1...",How many lines is Fleh Way on?,"MATCH (var1) WHERE var1.id=""7e8b034d-2480-4b47..."
1,"[{'architecture': 'victorian', 'cleanliness': ...","[{'line_color': 'gray', 'line_id': 'cd3d211c-a...",How clean is Kem Boulevard?,"MATCH (var1) WHERE var1.id=""ccc4d443-c6f9-4874..."
2,"[{'architecture': 'art-deco', 'cleanliness': '...","[{'line_color': 'gray', 'line_id': 'cb4e9908-c...",Which stations does Blue Crusp pass through?,NaN
3,"[{'architecture': 'victorian', 'cleanliness': ...","[{'line_color': 'cyan', 'line_id': '4d596a65-3...",Which stations does Brown Spunk pass through?,NaN
4,"[{'architecture': 'concrete', 'cleanliness': '...","[{'line_color': 'orange', 'line_id': '5def3278...",How many victorian stations are on the Green F...,NaN


In [ ]:
lol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8032 entries, 0 to 8031
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   nodes    8032 non-null   object
 1   edges    8032 non-null   object
 2   english  8032 non-null   object
 3   cypher   3202 non-null   object
dtypes: object(4)
memory usage: 251.1+ KB


In [ ]:
lol_df.dropna(inplace=True)
lol_df.reset_index(inplace=True, drop=True)
lol_df.shape

(3202, 4)

In [ ]:
data = []
for i in range(0, lol_df.shape[0]):
    data.append(#lol_df["nodes"][i] + "\n" + lol_df["edges"][i] + "\n" +
                lol_df["english"][i] + "\n" + lol_df["cypher"][i])

data[0]

'How many lines is Fleh Way on?\nMATCH (var1) WHERE var1.id="7e8b034d-2480-4b47-8892-288a8ba9af6d"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  WITH length(var4)  AS var5 RETURN var5'

# Transform Data

In [ ]:
len(data)

3202

In [ ]:
train_dataset = data[:2702]
val_dataset = data[2702:3102]
test_dataset = data[3102:]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

# train_dataset = data['validation'].select(range(100)).map(lambda x: {"input_text": x['question']  + "\n" + x['best_answer']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset, truncation=True, padding=True, max_length=256, return_tensors='pt')
val_encodings = tokenizer(val_dataset, truncation=True, padding=True, max_length=256, return_tensors='pt')
test_encodings = tokenizer(test_dataset, truncation=True, padding=True, max_length=256, return_tensors='pt')

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
# Convert the encodings to PyTorch datasets
train = TextDataset(train_encodings)
val = TextDataset(val_encodings)
test = TextDataset(test_encodings)

In [ ]:
len(train)

2702

In [ ]:
len(val)

400

In [ ]:
len(test)

100

# Example Before Fine Tuning

In [ ]:
def generate(index):

  example_text = lol_df['english'][index]
  correct_answer = lol_df['cypher'][index]

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print("Best Answer:")
  print(correct_answer)

  print()

In [ ]:
generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
How many lines is Fleh Way on?
Answer:
How many lines is Fleh Way on?
I'm sorry, I cannot provide an accurate answer as Fleh Way is not a known term or location. Can you please provide more context or information?
Best Answer:
MATCH (var1) WHERE var1.id="7e8b034d-2480-4b47-8892-288a8ba9af6d"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  WITH length(var4)  AS var5 RETURN var5



# Training

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    args=transformers.TrainingArguments(
        num_train_epochs=30,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

<ipython-input-35-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.046000
2,3.014200
3,2.954500
4,3.041000
5,2.959200
6,3.000800
7,2.862000
8,2.815500
9,2.611300
10,2.654600


TrainOutput(global_step=100, training_loss=1.468735535144806, metrics={'train_runtime': 2778.1491, 'train_samples_per_second': 2.304, 'train_steps_per_second': 0.036, 'total_flos': 1.90997027799552e+16, 'train_loss': 1.468735535144806, 'epoch': 2.37})

# Example After Fine Tuning

In [ ]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_

In [ ]:
# from train set
generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
How many lines is Fleh Way on?
Answer:
How many lines is Fleh Way on?
MATCH (var1) WHERE var1.id="f7b7b7b7-b0a4-4b0b-b7d4-b7c7d7c7c7f"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS
Best Answer:
MATCH (var1) WHERE var1.id="7e8b034d-2480-4b47-8892-288a8ba9af6d"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  WITH length(var4)  AS var5 RETURN var5



In [ ]:
# from validation set
generate(3050)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Which lines is Hink Court on?
Answer:
Which lines is Hink Court on?
MATCH (var1) WHERE var1.id="f7b7b7b7-b0a4-4b0b-b7d7-b7c7d7c7f7d"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS
Best Answer:
MATCH (var1) WHERE var1.id="80d01f67-54c3-4088-9be7-f8203b548520"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  RETURN var4



In [ ]:
# from test set
generate(3150)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
How many lines is Mebton on?
Answer:
How many lines is Mebton on?
MATCH (var1) WHERE var1.id="f7b7b7b7-b0a4-4b0b-b7d7-b7c7d7c7c7f"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS
Best Answer:
MATCH (var1) WHERE var1.id="713c2c94-3ec4-4591-8cdb-da05ea291800"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  WITH length(var4)  AS var5 RETURN var5



# Evaluate

[Learn the ROUGE metric](https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499#:~:text=ROUGE%2D1%20recall%20can%20be,number%20of%20unigrams%20in%20R.&text=Then%2C%20ROUGE%2D1%20F1%2D,the%20standard%20F1%2Dscore%20formula.)

[Code](https://github.com/pltrdy/rouge)

In [ ]:
list(lol_df[3102:]['cypher'])[0]

'MATCH (var1) WHERE var1.id="fd30d669-367e-4fee-b1f2-06b49003b89e"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  RETURN var4'

In [ ]:
def predict(index):

  example_text = lol_df['english'][index]
  correct_answer = lol_df['cypher'][index]

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=256, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  pred = tokenizer.decode(output[0], skip_special_tokens=True)
  # pred = "\n".join(pred.split('\n')[1:])
  pred = pred.split('\n')[1]
  return(pred)

In [ ]:
generate(3050)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Which lines is Hink Court on?
Answer:
Which lines is Hink Court on?
MATCH (var1) WHERE var1.id="f7b7b7b7-b0a4-4b0b-b7d7-b7c7d7c7f7d"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS
Best Answer:
MATCH (var1) WHERE var1.id="80d01f67-54c3-4088-9be7-f8203b548520"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  RETURN var4



In [ ]:
predict(3050)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'MATCH (var1) WHERE var1.id="f7b7b7b7-b0a4-4b0b-b7d7-b7c7d7c7f7d"  MATCH (var1)-[tmp1]-() WITH var1, collect(tmp1) AS var2 UNWIND var2 as tmp2 WITH var1, var2, collect(tmp2.line_name) AS var3  UNWIND var3 as tmp3 WITH DISTINCT tmp3 as tmp4 WITH collect(tmp4) as var4  RETURN var4 AS var5   WITH var5 as tmp5  AS var6 WITH collect(tmp6) as var7  RETURN var7 AS var8   WITH var8 as tmp8  AS var9 WITH collect(tmp9) as var10  RETURN var10   AS var11   WITH var11 as tmp11  AS var12   WITH var12 as tmp12  RETURN var12   AS var13   WITH var13 as tmp13  AS var14   WITH var14 as tmp14  RETURN var14   AS var15   WITH var15 as tmp15'

In [ ]:
import numpy as np

predicted = []
for i in np.arange(3102, 3202):
    predicted.append(predict(i))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

In [ ]:
y_test = []
for test_cypher in list(lol_df[3102:]['cypher']):
    y_test.append(test_cypher.split('\n')[0])

In [ ]:
len(predicted), len(y_test)

In [ ]:
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(predicted, y_test, avg=True)

In [ ]:
scores

In [ ]:
scores

{'rouge-1': {'r': 0.5155641025641021,
  'p': 0.8270364145658249,
  'f': 0.6330194373566892},
 'rouge-2': {'r': 0.34782936507936485,
  'p': 0.6374822874493921,
  'f': 0.4489808766626945},
 'rouge-l': {'r': 0.5155641025641021,
  'p': 0.8270364145658249,
  'f': 0.6330194373566892}}

# Save model locally

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("/content/drive/MyDrive/cypher_Q&A/falcon-7b-finetuned")

In [ ]:
lora_config = LoraConfig.from_pretrained('/content/drive/MyDrive/cypher_Q&A/falcon-7b-finetuned')
model = get_peft_model(model, lora_config)


In [ ]:
text = "We have a graph-based dataset. Which consists of 2 types of nodes: Person (Properties: name, born), Movie (Properties: tagline, title, released), and 6 types of relationships: FOLLOWS, ACTED_IN (Properties: roles), DIRECTED, PRODUCED, WROTE, REVIEWED (Properties: summary, rating). Generate cypher query for this graph database from the given sentences. Find total number of movies."
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


We have a graph-based dataset. Which consists of 2 types of nodes: Person (Properties: name, born), Movie (Properties: tagline, title, released), and 6 types of relationships: FOLLOWS, ACTED_IN (Properties: roles), DIRECTED, PRODUCED, WROTE, REVIEWED (Properties: summary, rating). Generate cypher query for this graph database from the given sentences. Find total number of movies.
MERGE (PERSON, {name: "Person", born: 


In [ ]:
text = "We have a graph-based dataset. Which consists of 2 types of nodes: Person (Properties: name, born), Movie (Properties: tagline, title, released), and 6 types of relationships: FOLLOWS, ACTED_IN (Properties: roles), DIRECTED, PRODUCED, WROTE, REVIEWED (Properties: summary, rating). Generate cypher query for this graph database from the given sentences. Find total number of movies."
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


We have a graph-based dataset. Which consists of 2 types of nodes: Person (Properties: name, born), Movie (Properties: tagline, title, released), and 6 types of relationships: FOLLOWS, ACTED_IN (Properties: roles), DIRECTED, PRODUCED, WROTE, REVIEWED (Properties: summary, rating). Generate cypher query for this graph database from the given sentences. Find total number of movies.
WITH (p:Person) AS 'Person', m:Movie (p


# Merge Adapters and Save Model to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# model.save_pretrained("falcon-7b-instruct-ft-adapters", push_to_hub=True) # save LORA adapters
model.push_to_hub("falcon-7b-instruct-ft-adapters", use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NilavoBoral/falcon-7b-instruct-ft-adapters/commit/83eead4c2209d1fa4aa7b30a8196533aae8cbac6', commit_message='Upload model', commit_description='', oid='83eead4c2209d1fa4aa7b30a8196533aae8cbac6', pr_url=None, pr_revision=None, pr_num=None)